<a href="https://colab.research.google.com/github/imtheguna/PySpark-Learning/blob/GoogleColab/9_PySpark_GroupBy().ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()
!pip install pyspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [830 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,069 kB]
Get:13 http://archive.

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession \
      .builder \
      .appName('SelectColumns').getOrCreate()

df = spark.read.csv('/content/data2.csv',header=True,inferSchema=True)

df.show(1)

+-------+----------------+-----------+-----------+-------------------+----------------+------------------+--------------+----------------------+-------------+---------------+
| period|series_reference|region_name|filled jobs|filled jobs revised|filled jobs diff|filled jobs % diff|total_earnings|total earnings revised|earnings diff|earnings % diff|
+-------+----------------+-----------+-----------+-------------------+----------------+------------------+--------------+----------------------+-------------+---------------+
|2020.09|     BDCQ.SED1RA|  Northland|      65520|              65904|             384|               0.6|           953|                   959|            6|            0.6|
+-------+----------------+-----------+-----------+-------------------+----------------+------------------+--------------+----------------------+-------------+---------------+
only showing top 1 row



In [7]:
df1 = df.groupBy(['region_name']).agg(sum('total_earnings').alias('total_earnings_region_name'))

df1.show(5)

+-----------+--------------------------+
|region_name|total_earnings_region_name|
+-----------+--------------------------+
| Wellington|                     46619|
|Hawke's Bay|                     11483|
|   Auckland|                    135460|
|   Gisborne|                      3019|
|  Southland|                      6740|
+-----------+--------------------------+
only showing top 5 rows



In [13]:
df1 = df.where(col('total_earnings')>500) \
      .groupBy(['region_name']) \
      .agg(sum('total_earnings').alias('total_earnings_region_name') ,
           sum('total earnings revised').alias('total_earnings_revised_region_name'))


df1.orderBy('region_name').show(5)

+------------------+--------------------------+----------------------------------+
|       region_name|total_earnings_region_name|total_earnings_revised_region_name|
+------------------+--------------------------+----------------------------------+
|          Auckland|                    135460|                            137064|
|     Bay of Plenty|                     20843|                             21077|
|        Canterbury|                     45601|                             46080|
|       Hawke's Bay|                     11483|                             11591|
|Manawatu-Whanganui|                     15693|                             15801|
+------------------+--------------------------+----------------------------------+
only showing top 5 rows

